# Numerical Methods for Manu Body Physics, Assignment #2

Yoav Zack, ID 211677398

## Question 1

We want to calculate the entanglement entropy of a random state under the $A\otimes B$ partition. A random state of the whole system is:
$$\ket{\psi} = \sum_{k} c_{k} \ket{k}$$
But under the bi-partition $A\otimes B$  a state is of the following form:
$$\ket{\psi} = \sum_{i,j} \Psi_{i,j} \ket{i} \ket{j}$$
Where $\ket{i}, \ket{j}$ are elements in a basis for $A$ and $B$ respectively. So, to get such a representation of the state  we will write:
$$\sum_{k} c_{k} \ket{k} = \sum_{i,j} \Psi_{i,j} \ket{i} \ket{j}$$
Assuming the original basis $\ket{k}$ is the standard spin basis and the new ones $\ket{i}, \ket{j}$ are the same but for the respective subsystems, we can write each state $\ket{k}$ as $\ket{i} \otimes \ket{j}$, where:
$$i=\sum_{n=1}^{L/2} 2^{k[2n]}\; ; \; i=\sum_{n=1}^{L/2} 2^{k[2n-1]}$$
where $k[n]$ is the $n$-th bit of the number $k$. Using this, we can write an expression for the matrix $\Psi$ as $\Psi_{i,j} = c_k$ assuming $i,j$ are defined as above.

After all of this is done, we can calculate the Von Neumann Entanglemnt entopy of the system by performing a Schmidt Decomposition (i.e. SVD) on the matrix $\Psi_{ij}$, an summing the singular values $\lambda_i$ in the following form:
$$S = -\sum_{i}\lambda_{i} \log \lambda_{i}$$

Let's do that in `julia`. First we import useful libraries:

In [ ]:
using LinearAlgebra
using Plots

Then we define the bipartition of the wavefunction into two staggared chains:

In [ ]:
function index2state(stateind::Integer, N::Integer)
  return digits(stateind, base=2, pad=N)
end

In [ ]:
function staggared_bipartition(ψ::Vector{<:Number})
    @assert abs(norm(ψ) - 1) < 1e-9 "State not normalized"
    D = length(ψ)
    L = log2(D)
    @assert isinteger(L) "Number of states must be a power of two"
    L = Int(L)
    @assert isinteger(L/2) "Number of spins must be even"

    r = sqrt(D)
    @assert isinteger(r) "Number of states must be a square number"
    r = Int(r)
    
    Ψ = zeros(r, r)
    for k in range(0, length=D)
        i = sum([(typeof(k)(1) << (2*n  ) & k) >> (n  ) for n in range(0, length=Int(L/2))])
        j = sum([(typeof(k)(1) << (2*n+1) & k) >> (n+1) for n in range(0, length=Int(L/2))])
        # @show index2state(k, L), index2state(i, L), index2state(j, L)
        # @show k, i, j
        Ψ[i+1,j+1] = ψ[k+1]
    end
    return Ψ
end

And we define function which takes a wavefunction in the form of a vector and a bipartition function, performs the bipartition and calcualtes the Von-Neumann Entropy for that bypartition:

In [ ]:
function vn_entropy(ψ::Vector{<:Number}, bipartition::Function)
    Ψ = bipartition(ψ)
    return vn_entropy(Ψ)
end

function vn_entropy(Ψ::Matrix{<:Number})
    S = svdvals(Ψ)
    return sum(-S[S.!=0].^2 .* log.(S[S.!=0].^2))
end

We can test this function on all of the Bell States:

In [ ]:
@show vn_entropy([1,0,0,1]/sqrt(2), staggared_bipartition);
@show vn_entropy([1,0,0,-1]/sqrt(2), staggared_bipartition);
@show vn_entropy([0,1,1,0]/sqrt(2), staggared_bipartition);
@show vn_entropy([0,1,-1,0]/sqrt(2), staggared_bipartition);
@show log(2);

And as can be seen, all of them are equal to $\ln 2$ as expected. We can also test this on a simple case of 10,000 random states on 10 spins:

In [ ]:
N = 10000
L = 4
svn = zeros(N)
for iter in range(1, length=N)
    ψ = normalize(rand(2^L))
    svn[iter] = vn_entropy(ψ, staggared_bipartition)
end
histogram(svn, legend=nothing, xlabel="Entanglement Entopy", ylabel="Count")